In [12]:
import livy, pprint, requests
host = livy.getHostUrl("http://34.205.23.216:9191")

In [13]:
session_url = livy.openSession(host)

..........................

In [14]:
upload_file_url = session_url + "/upload-file"

In [15]:
requests.post(upload_file_url, files={"file": open("gtex_rnaseq_prep_profiles.tsv","rb")})

<Response [200]>

In [16]:
code = '''
from pyspark import SparkContext
from pyspark import SparkFiles
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

path = SparkFiles.get("gtex_rnaseq_prep_profiles.tsv")

dataheaderindex = 2

with open(path) as inp:
    headers = inp.readline().strip().split("\t")
    # ids = []
    # features = []
    data = []
    for line in inp:
        row = line.strip().split('\t')
        if len(row) < dataheaderindex:
            continue
        # id = row[:dataheaderindex]
        feature = map(float, row[dataheaderindex:])
        data.append(feature)
    data2 = list(map(lambda x : [Vectors.dense(*x)], zip(*data)))

df = spark.createDataFrame(data2, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\\n" + str(r1))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\\n" + str(r2))
'''


In [17]:
r = livy.execStatement(host, session_url, code)

..........

In [18]:
print(r.json())

{'code': '\nfrom pyspark import SparkContext\nfrom pyspark import SparkFiles\nfrom pyspark.ml.linalg import Vectors\nfrom pyspark.ml.stat import Correlation\n\npath = SparkFiles.get("gtex_rnaseq_prep_profiles.tsv")\n\ndataheaderindex = 2\n\nwith open(path) as inp:\n    headers = inp.readline().strip().split("\t")\n    # ids = []\n    # features = []\n    data = []\n    for line in inp:\n        row = line.strip().split(\'\t\')\n        if len(row) < dataheaderindex:\n            continue\n        # id = row[:dataheaderindex]\n        feature = map(float, row[dataheaderindex:])\n        data.append(feature)\n    data2 = list(map(lambda x : [Vectors.dense(*x)], zip(*data)))\n\ndf = spark.createDataFrame(data2, ["features"])\n\nr1 = Correlation.corr(df, "features").head()\nprint("Pearson correlation matrix:\\n" + str(r1))\n\nr2 = Correlation.corr(df, "features", "spearman").head()\nprint("Spearman correlation matrix:\\n" + str(r2))\n\nr3 = Correlation.corr(df, "features", "tanimoto").head

In [8]:
pprint.pprint(r.json()["output"]["data"])

{'text/plain': 'Pearson correlation matrix:\n'
               'Row(pearson(features)=DenseMatrix(9, 9, [1.0, 0.2447, 0.2068, '
               '0.4914, 0.3069, -0.0747, 0.1278, 0.3957, ..., 0.0188, 0.2582, '
               '0.5914, 0.3462, 0.1979, 0.0447, 0.3077, 1.0], False))\n'
               'Spearman correlation matrix:\n'
               'Row(spearman(features)=DenseMatrix(9, 9, [1.0, 0.223, 0.3006, '
               '0.4801, 0.3286, 0.3539, 0.37, 0.5528, ..., 0.1393, 0.2525, '
               '0.4988, 0.3717, 0.4857, 0.1324, 0.3615, 1.0], False))'}


In [9]:
livy.closeSession(session_url)